# Numerical Linear Algebra [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ua-2025q3-astr501-513/ua-2025q3-astr501-513.github.io/blob/main/513/02/notes.ipynb)

[![Matrix transform](fig/matrix_transform.png)](https://xkcd.com/184/)

```{note} TAP Computation and Data Intuitive Meeting

Date: Every Thursday  
Time: 2-3pm  
Room: SO N305  
Zoom: [one-click](https://arizona.zoom.us/j/88694275321?pwd=XiFa1kbUVl90MYtoAa47W6FCcuRowU.1), id: 886 9427 5321, password: tapcdi  
Schedule: [Google Sheet](https://docs.google.com/spreadsheets/d/1VQkQGZYwSEJ_N6UIHJQ-Tjvn02k9rClImgCCYo4ucrg/edit?usp=sharing)

Upcoming topic: "Book keeping of your simulations (or large data sets)"
```

```{note} HPC Workshop

UA HPC provides HPC workshop during this Fall:

| Date | Time | Session
--- | --- | ---
Friday Sep 12th | 10am-3pm | Introduction to HPC
Friday Sep 19th | 10am-3pm | Software on HPC
Friday Sep 26th | 10am-3pm | Machine Learning and GPUs

Register with this
[Google Form](https://docs.google.com/forms/d/e/1FAIpQLSfjRhn1xF7wcd6G_wyVKtdYqosxxPaM_2V-nfTJZa8BXEe5lA/viewform).
```

```{admonition} Homework Set #1

Use this GitHub Classroom Link:
https://classroom.github.com/a/r-eqz-mO
to accept it.

Please make sure you merge from the upstream repository so all the
autograding and template are in place.
```

Linear algebra is a fundamental part of modern mathematics.
It supports fields from calculus and statistics to geometry, control
theory, and functional analysis.
Most linear systems are well understood.
Even nonlinear problems are often studied through linear
approximations.

Numerical linear algebra extends these ideas to computation, enabling
solutions of PDEs, optimization tasks, eigenvalue problems, and
more.
It addressing some of the hardest problems in physics and engineering.

### Motivations from Physics

* Normal Modes:
  Vibrations near equilibrium reduce to generalized eigenvalue
  problems.
  Linear algebra therefore reveals resonance in materials, acoustics,
  and plasma waves.

* Quantum Mechanics:
  Described by the Schrödinger equation, quantum systems are
  inherently linear.

* Discretized PDEs:
  Discretizing PDEs yields large sparse linear systems.
  They can solved numerically by methods such as conjugate gradient.

* Nonlinear Problems:
  Nonlinear physics problems including turbulence are sometimes
  untrackable.
  Linearizing them with perturbation theory reduces them to sequences
  of linear systems.

### Motivations from Computation

* Large-Scale Data:
  Modern sensors and simulations produce massive datasets.
  Matrix decompositions (e.g., SVD, PCA) provide compression, noise
  reduction, and feature extraction.

* Neural Networks:
  Core operations in training, i.e., backpropagation, is dominated by
  large matrix multiplications.
  Efficient linear algebra routines are therefore critical for scaling
  deep learning.

* Hardware Accelerators:
  GPUs and TPUs are optimized for matrix operations, making vectorized
  linear algebra essential for both neural networks and scientific
  computing.

```{note}

> If everything were linear, we wouldn't need computers.
> <div style="text-align: right">- Arrogant mathematicians, including CK many years ago ...</div>

The above quote suggests that in a purely linear world, everything
would be easy to solve analytically.
However, this is an oversimplification.

Even perfectly linear problems can pose significant computational
challenges due to two key factors.
* High dimensionality makes solving linear systems computationally
  intensive.
  For example, systems with millions of unknowns are common in
  numerical PDEs or massive machine learning models.
  Processing such large-scale data requires significant computational
  power, regardless of linearity.
* Real-world computations face constraints from finite precision.
  Hardware limitations, such as floating-point arithmetic, introduce
  numerical stability and conditioning challenges, even in linear
  systems.
  Addressing these issues requires robust algorithms to ensure
  accurate and efficient solutions.

Some of the most exciting development in numerical analysis recently
is to apply randomized algorithms to solve large scale linear algebra
problems.
See [this reference](https://arxiv.org/pdf/2402.17873) for an
introductory course.
```

## Direct Solvers

Direct methods are often the first approach taught for solving linear
systems $A\mathbf{x} = \mathbf{b}$.
They involve algebraic factorizations that can be computed in a fixed
number of steps (roughly $\mathcal{O}(n^3)$) for an $n \times n$
matrix.

### Gaussian Elimination

Gaussian Elimination transforms the system $A \mathbf{x} = \mathbf{b}$
into an equivalent upper-triangular form $U \mathbf{x} = \mathbf{c}$
by systematically applying row operations.  Once in an
upper-triangular form, one can perform back-substitution to solve for
$\mathbf{x}$.

1. Row Operations
   * Subtract a multiple of one row from another to eliminate entries
     below the main diagonal.
   * Aim to create zeros in column $j$ below row $j$.

2. Partial Pivoting (optional)
   * When a pivot (diagonal) element is small (or zero), swap the
     current row with a row below that has a larger pivot element in
     the same column.
   * This step mitigates numerical instability by reducing the chance
     that small pivots lead to large rounding errors in subsequent
     operations.

3. Result
   * After eliminating all sub-diagonal entries, the matrix is in
     upper-triangular form $U$.
   * Solve $U\mathbf{x} = \mathbf{c}$ via back-substitution.


Here is an
[example](https://en.wikipedia.org/wiki/Gaussian_elimination):

```{list-table}
:header-rows: 1
* + System of equations
  + Row operations
  + Augmented matrix

* + \begin{alignat}{4}
       2x &{}+{}& y &{}-{}&  z &{}={}&   8 & \\
      -3x &{}-{}& y &{}+{}& 2z &{}={}& -11 & \\
      -2x &{}+{}& y &{}+{}& 2z &{}={}&  -3 &
    \end{alignat}
  +
  + \begin{align}
    \left[\begin{array}{rrr|r}
       2 &  1 & -1 &   8 \\
      -3 & -1 &  2 & -11 \\
      -2 &  1 &  2 &  -3
    \end{array}\right]
    \nonumber
    \end{align}

* + \begin{alignat}{4}
      2x &{}+{}&          y &{}-{}&          z &{}={}& 8 & \\
         &     & \tfrac12 y &{}+{}& \tfrac12 z &{}={}& 1 & \\
         &     &         2y &{}+{}&          z &{}={}& 5 &
    \end{alignat}
  + \begin{align}
      L_2 + \tfrac32 L_1 &\to L_2 \\
      L_3 +          L_1 &\to L_3
    \end{align}
  + \begin{align}
    \left[\begin{array}{rrr|r}
      2 &      1  &     -1  & 8 \\
      0 & \frac12 & \frac12 & 1 \\
      0 &      2  &      1  & 5
    \end{array}\right]
    \end{align}

* + \begin{alignat}{4}
      2x &{}+{}&          y &{}-{}&          z &{}={}& 8 & \\
         &     & \tfrac12 y &{}+{}& \tfrac12 z &{}={}& 1 & \\
         &     &            &     &         -z &{}={}& 1 &
    \end{alignat}
  + \begin{align}
      L_3 + -4 L_2 \to L_3
    \end{align}
  + \begin{align}
    \left[\begin{array}{rrr|r}
      2 &      1  &     -1  & 8 \\
      0 & \frac12 & \frac12 & 1 \\
      0 &      0  &     -1  & 1
    \end{array}\right]
    \end{align}
```

The matrix is now in echelon form (also called triangular form):

```{list-table}
:header-rows: 1
* + System of equations
  + Row operations
  + Augmented matrix

* + \begin{alignat}{4}
      2x &{}+{}&          y &     &   &{}={}       7  & \\
         &     & \tfrac12 y &     &   &{}={} \tfrac32 & \\
         &     &            &{}-{}& z &{}={}       1  &
    \end{alignat}
  + \begin{align}
      L_1 -          L_3 &\to L_1\\
      L_2 + \tfrac12 L_3 &\to L_2
    \end{align}
  + \begin{align}
    \left[\begin{array}{rrr|r}
      2 &      1  &  0 &      7  \\
      0 & \frac12 &  0 & \frac32 \\
      0 &      0  & -1 &      1
    \end{array}\right]
    \end{align}

* + \begin{alignat}{4}
      2x &{}+{}& y &\quad&   &{}={}&  7 & \\
         &     & y &\quad&   &{}={}&  3 & \\
         &     &   &\quad& z &{}={}& -1 &
    \end{alignat}
  + \begin{align}
       2 L_2 &\to L_2 \\
      -L_3 &\to L_3
    \end{align}
  + \begin{align}
    \left[\begin{array}{rrr|r}
      2 & 1 & 0 &  7 \\
      0 & 1 & 0 &  3 \\
      0 & 0 & 1 & -1
    \end{array}\right]
    \end{align}

* + \begin{alignat}{4}
      x &\quad&   &\quad&   &{}={}&  2 & \\
        &\quad& y &\quad&   &{}={}&  3 & \\
        &\quad&   &\quad& z &{}={}& -1 &
    \end{alignat}
  + \begin{align}
               L_1 - L_2 &\to L_1 \\
      \tfrac12 L_1       &\to L_1
    \end{align}
  + \begin{align}
    \left[\begin{array}{rrr|r}
      1 & 0 & 0 &  2 \\
      0 & 1 & 0 &  3 \\
      0 & 0 & 1 & -1
    \end{array}\right]
    \end{align}
```

Below is a simple code for naive (no pivoting) Gaussian Elimination in
python.
Although normally we want to avoid for loops in python for
performance, let's stick with for loop this time so we can directly
implement the algorithm we just described.

In [ ]:
import numpy as np

def solve_Gaussian(A, b):
    """
    Perform naive (no pivoting) Gaussian elimination to solve the
    matrix equation A x = b.
    Returns the solution vector x.
    """
    assert A.ndim == 2 and A.shape[0] == A.shape[1]  # must be square matrix
    assert b.ndim == 1 and b.shape[0] == A.shape[1]  # must be a vector
    
    A = A.astype(float)  # ensure floating-point, create copy by default
    b = b.astype(float)  # ensure floating-point, create copy by default
    n = b.shape[0]

    # Forward elimination
    for k in range(n-1):
        for i in range(k+1, n):
            if A[k, k] == 0:
                raise ValueError("Zero pivot encountered (no pivoting).")
            factor    = A[i, k] / A[k, k]
            A[i, k:] -= factor * A[k, k:]
            b[i]     -= factor * b[k]

    # Back-substitution
    x = np.zeros(n)
    for i in reversed(range(n)):
        s = b[i]
        for j in range(i+1, n):
            s -= A[i, j] * x[j]
        x[i] = s / A[i, i]

    return x

Let's also compare with numpy's solver.

In [ ]:
A = np.random.random((3, 3))
b = np.random.random((3))

x_numpy = np.linalg.solve(A, b)
x_naive = solve_Gaussian(A, b)

print(x_numpy, "numpy")
print(x_naive, "naive")
print(abs(x_naive - x_numpy), "difference with niave")

Let's set the (0,0) element to a small value.

In [ ]:
A[0,0] = 1e-16

x_numpy = np.linalg.solve(A, b)
x_naive = solve_Gaussian(A, b)

print(x_numpy, "numpy")
print(x_naive, "naive")
print(abs(x_naive - x_numpy), "difference with niave")

Let's now improve the above naive (no pivoting) Gaussian elimination by adding pivoting.

In [ ]:
# HANDSON: improve the above naive (no pivoting) Gaussian elimination
#          by adding pivoting.

def solve_Gaussian_pivot(A, b):
    """
    Perform Gaussian elimination with partial pivoting to solve
    the matrix equation A x = b.
    Returns the solution vector x.
    """
    assert A.ndim == 2 and A.shape[0] == A.shape[1]  # must be square matrix
    assert b.ndim == 1 and b.shape[0] == A.shape[1]  # must be a vector
    
    A = A.astype(float)  # ensure floating-point, create copy by default
    b = b.astype(float)  # ensure floating-point, create copy by default
    n = b.shape[0]

    # Forward elimination
    for k in range(n-1):
        # TODO: pivoting: find max pivot in column k

        # TODO: swap rows if needed
        
        for i in range(k+1, n):
            ### No longer a problem
            # if A[k, k] == 0:
            #     raise ValueError("Zero pivot encountered (no pivoting).")
            factor    = A[i, k] / A[k, k]
            A[i, k:] -= factor * A[k, k:]
            b[i]     -= factor * b[k]

    # Back-substitution
    x = np.zeros(n)
    for i in reversed(range(n)):
        s = b[i]
        for j in range(i+1, n):
            s -= A[i, j] * x[j]
        x[i] = s / A[i, i]

    return x

In [ ]:
x_naive = solve_Gaussian(A, b)
x_pivot = solve_Gaussian_pivot(A, b)
x_numpy = np.linalg.solve(A, b)

print(x_numpy, "numpy")
print(x_naive, "naive")
print(x_pivot, "pivot")
print(abs(x_naive - x_numpy), "difference with niave")
print(abs(x_pivot - x_numpy), "difference with pivot")

In [ ]:
# HANDSON:
#
# Change A and b to larger matrices.
# Again set A[0,0] to a small value.
#
# Test how solve_Gaussian() and solve_Gaussian_pivot() perform
# compared to numpy.linalg.sovle().


### $LU$ Decomposition

$LU$ Decomposition is a systematic way to express $A$ as $A = P L U$,
where:
* $P$ is a permutation matrix
* $L$ is lower triangular (with 1s on the diagonal following the
  standard convention).
* $U$ is upper triangular.

Once $A$ is factored as $P L U$, solving $A\mathbf{x} = \mathbf{b}$
becomes:
1. $L \mathbf{y} = P^t \mathbf{b}$ (forward substitution)
2. $U \mathbf{x} =     \mathbf{y}$ (back-substitution)

Gaussian elimination essentially constructs the $L$ and $U$ matrices
behind the scenes:
* The multipliers used in the row operations become the entries of
  $L$.
* The final upper-triangular form is $U$.

In [ ]:
import scipy.linalg as la

A = np.random.random((3, 3))
b = np.random.random((3))

# Perform LU decomposition with pivoting
P, L, U = la.lu(A)  # P is the permutation matrix

print("Permutation matrix P:")
print(P)
print("Lower triangular matrix L:")
print(L)
print("Upper triangular matrix U:")
print(U, end="\n\n")

# Forward substitution for L y = Pt b
y = la.solve_triangular(L, np.dot(P.T, b), lower=True, unit_diagonal=True)
# Back substitution for U x = y
x_LU = la.solve_triangular(U, y)

print("Solution using LU decomposition:", x_LU)
print("Check with np.linalg.solve(A, b):", np.linalg.solve(A, b))

In [ ]:
# HANDSON:
#
# Keep A at 3x3 but set A[0,0] to a small value.
# How does scipy.linalg.lu()'s solution compared to
# numpy.linalg.sovle()?
#
# Change A and b to larger matrices, and again set A[0,0] to a small
# value.
# How does scipy.linalg.lu()'s solution compared to
# numpy.linalg.sovle()?


## Matrix Inverse

Although the matrix inverse $A^{-1}$ is a central theoretical concept,
explicitly forming $A^{-1}$ just to solve $A \mathbf{x} = \mathbf{b}$
is almost always unnecessary and can degrade numerical stability.
Instead, direct approaches (like Gaussian Elimination or LU
factorization) find $\mathbf{x}$ with fewer operations and less error
accumulation.
Both methods cost about $\mathcal{O}(n^3)$, but computing the entire
inverse introduces extra steps and can magnify floating-point errors.

In rare cases, you might actually need $A^{-1}$.

For example, when:
* Multiple Right-Hand Sides:
  If you must solve $A \mathbf{x} = \mathbf{b}_i$ for many different
  $\mathbf{b}_i$, you might form or approximate $A^{-1}$ for
  convenience.
* Inverse-Related Operations:
  Some advanced algorithms (e.g., computing discrete Green's functions
  or certain control theory design methods) explicitly require
  elements of the inverse.

In the demonstration below, we show how to compute the inverse using
`np.linalg.inv()`.
However, again, it is generally safer and more efficient to solve
individual systems via a factorization method in practice.

In [ ]:
# Simple 2x2 matrix
A = np.array([[2.0, 1.0],
              [1.0, 2.0]], dtype=float)

# Compute inverse using NumPy
A_inv = np.linalg.inv(A)
print("Inverse of A:")
print(A_inv)

# Verify A_inv is indeed the inverse
I_check = A @ A_inv
print("Check A * A_inv == I?")
print(I_check)

In [ ]:
# HANDSON:
#
# Construct a larger matrix, maybe with `numpy.random.random()`,
# and then use `numpy.linalg.inv()` to compute its inverse.
# Check the inversion actually works.


In [ ]:
# HANDSON:
#
# Compare the numerical solution of Ax = b by using
# `numpy.linalg.solve()` and by using `numpy.linalg.inv()` and then by
# matrix multiplication.


## Iterative Solvers for Large/Sparse Systems

Direct methods are powerful but quickly become impractical when
dealing with very large or sparse systems.
In such cases, iterative solvers provide a scalable alternative.

* Large-Scale Problems:
  PDE discretizations in 2D/3D can easily lead to systems with
  millions of unknowns, making $\mathcal{O}(n^3)$ direct approaches
  infeasible.
* Sparse Matrices:
  Many physical systems produce matrices with a significant number of
  zero entries.
  Iterative methods may be implemented to access only nonzero elements
  each iteration, saving time and memory.
* Scalability:
  On parallel architectures (clusters, GPUs, etc), iterative methods
  often scale better than direct factorizations.

### Jacobi Iteration

1. **Idea:** Rewrite $A\mathbf{x} = \mathbf{b}$ as $\mathbf{x} = D^{-1}(\mathbf{b} - R\mathbf{x})$, where $D$ is the diagonal of $A$ and $R$ is the remainder.  
2. **Update Rule:**
   \begin{align}
     x_i^{(k+1)} = \frac{1}{a_{ii}} \Big(b_i - \sum_{j \neq i} a_{ij} x_j^{(k)}\Big).
   \end{align}
3. **Pros/Cons:**
   - **Easy to implement**; each iteration updates $\mathbf{x}$ using only values from the previous iteration.  
   - **Slow convergence** unless $A$ is well-conditioned (e.g., diagonally dominant).

### Jacobi Iteration

1. Idea:
   Rewrite
   $A\mathbf{x} = \mathbf{b}$ as $\mathbf{x} = D^{-1}(\mathbf{b} - R\mathbf{x})$,
   where $D$ is the diagonal of $A$ and $R$ is the remainder.

2. Update Rule:
   \begin{align}
     x_i^{(k+1)} = \frac{1}{a_{ii}} \Big(b_i - \sum_{j \neq i} a_{ij} x_j^{(k)}\Big).
   \end{align}

3. Pros/Cons:
   * Easy to implement: each iteration updates $\mathbf{x}$ using only
     values from the previous iteration.
   * Slow convergence unless $A$ is well-conditioned (e.g., diagonally
     dominant).

In [ ]:
def solve_Jacobi(A, b, max_iter=1000, tol=1e-8):
    """
    Solve A x = b using Jacobi iteration.
    A is assumed to be square with non-zero diagonal.
    """
    assert A.ndim == 2 and A.shape[0] == A.shape[1]  # must be square matrix
    assert b.ndim == 1 and b.shape[0] == A.shape[1]  # must be a vector
    
    A = A.astype(float)  # ensure floating-point, create copy by default
    b = b.astype(float)  # ensure floating-point, create copy by default
    n = A.shape[0]

    # Jacobi iteration
    x = np.zeros(n)    
    for k in range(max_iter):
        x_old = np.copy(x)
        for i in range(n):
            # Sum over off-diagonal terms
            s = 0.0
            for j in range(n):
                if j != i:
                    s += A[i,j] * x_old[j]
            x[i] = (b[i] - s) / A[i, i]
        
        # Check for convergence
        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            print(f"Jacobi converged in {k+1} iterations.")
            return x
    
    print("Jacobi did not fully converge within max_iter.")
    return x

In [ ]:
# Example system (small, but let's pretend it's "sparse")
A = np.array([[4.0, -1.0, 0.0],
              [-1.0, 4.0, -1.0],
              [0.0, -1.0, 4.0]], dtype=float)
b = np.array([6.0, 6.0, 6.0], dtype=float)

x_jacobi = solve_Jacobi(A, b)
x_numpy  = np.linalg.solve(A, b)
print(x_jacobi, "Jacobi Interaction")
print(x_numpy,  "direction (numpy)")
print(abs(x_jacobi - x_numpy), "difference")

In [ ]:
# HANDSON:
#
# Change the tolerance by setting the keyword `tol` to different
# values.
# How does the numerical solution look like?
# How many more steps do it take to "converge" to the required
# tolerance level?


### Gauss-Seidel Iteration

Gauss-Seidel iteration is closely related to Jacobi but usually
converges faster, particularly if the matrix is strictly or diagonally
dominant.

1. Idea:
   * Instead of using only the old iteration values (from step $k$),
     Gauss-Seidel uses the most recent updates within the same
     iteration.
   * This often converges faster because you incorporate newly
     computed values immediately rather than waiting for the next
     iteration.

2. Update Rule:
   \begin{align}
     x_i^{(k+1)} = \frac{1}{a_{ii}} \Big( b_i - \sum_{j < i} a_{ij} x_j^{(k+1)} - \sum_{j > i} a_{ij} x_j^{(k)} \Big).
   \end{align}
   Note that $\mathbf{x}^{(k+1)}$ is already partially updated
   (for $j < i$).

3. Convergence Properties:
   * Gauss-Seidel is shown to converge for strictly diagonally
     dominant matrices, and often outperforms Jacobi in practice.
   * Still relatively slow for large, ill-conditioned systems.

In [ ]:
# HANDSON: update the Jacobi iteration code to Gauss-Seidel

def solve_GS(A, b, max_iter=1000, tol=1e-8):
    """
    Solve A x = b using Gauss-Seidel iteration.
    A is assumed to be square with non-zero diagonal.
    """
    assert A.ndim == 2 and A.shape[0] == A.shape[1]  # must be square matrix
    assert b.ndim == 1 and b.shape[0] == A.shape[1]  # must be a vector
    
    A = A.astype(float)  # ensure floating-point, create copy by default
    b = b.astype(float)  # ensure floating-point, create copy by default
    n = A.shape[0]

    # TODO: Gauss-Seidel iteration: what should be changed comared to Jacobi?
    x = np.zeros(n)    
    for k in range(max_iter):
        x_old = np.copy(x)
        for i in range(n):
            # Sum over off-diagonal terms
            s = 0.0
            for j in range(n):
                if j != i:
                    s += A[i,j] * x_old[j]
            x[i] = (b[i] - s) / A[i, i]
        
        # Check for convergence
        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            print(f"Jacobi converged in {k+1} iterations.")
            return x
    
    print("Jacobi did not fully converge within max_iter.")
    return x

In [ ]:
# HANDSON:
#
# Check that `solve_GS()` works as expected.


In [ ]:
# HANDSON:
#
# Change the tolerance by setting the keyword `tol` to different
# values.
# How does the numerical solution look like?
# How many more steps do it take to "converge" to the required
# tolerance level?
# How does this compared to Jacobi?


## Eigenvalue Problems

Eigenvalue problems show up in many physics applications, including
normal mode analysis, quantum mechanics, and stability analyses.

1. Normal Modes
   * Vibrational analyses of structures or molecules reduce to $K
     \mathbf{x} = \omega^2 M \mathbf{x}$.
     The solutions are eigenpairs $(\omega^2, \mathbf{x})$.
   * Each eigenvector $\mathbf{x}$ describes a mode shape; the
     eigenvalue $\omega^2$ is the squared frequency.

2. Quantum Mechanics
   * The Schrodinger equation in matrix form
     $\hat{H}|E\rangle = E|E\rangle$
     yields eigenvalues $E$ (energy levels) and eigenstates $|E\rangle$.
   * Collecting all eigenvectors (as columns) in a matrix
     diagonalizes $\hat{H}$ if it is Hermitian.

3. Stability Analysis
   * A linearized system near equilibrium produces a Jacobian $J$.
     Eigenvalues of $J$ show growth/decay rates of perturbations.
   * If real parts of eigenvalues are negative, the system is stable;
     if positive, it's unstable.

Given an $n \times n$ matrix $A$, the **eigenvalue problem** seeks
scalars $\lambda$ (eigenvalues) and nonzero vectors $\mathbf{v}$
(eigenvectors) such that:
\begin{align}
A \mathbf{v} = \lambda \mathbf{v}.
\end{align}
* **Eigenvalues** can be real or complex.
* **Eigenvectors** identify the directions in which $A$ acts as a
  simple scale transformation ($\lambda$).

### Power Method for the Dominant Eigenpair

The power method is a straightforward iterative technique for finding
the eigenpair associated with the largest-magnitude eigenvalue:

1. Algorithm:
   * Begin with an initial vector $\mathbf{x}^{(0)}$.
   * Apply $A$ repeatedly and normalize:
     \begin{align}
       \mathbf{x}^{(k+1)} = \frac{A \mathbf{x}^{(k)}}{\|A \mathbf{x}^{(k)}\|}.
     \end{align}
   * This converges to the eigenvector of the eigenvalue with the
     largest magnitude, assuming it's distinct.

2. Limitations
   * Only computes one eigenvalue/eigenvector.
   * Convergence can be slow if other eigenvalues have magnitude close
     to the dominant one.

In [ ]:
def eig_power(A, max_iter=1000, tol=1e-7):
    """
    Returns the eigenvalue of largest magnitude and its eigenvector.
    """
    assert A.ndim == 2 and A.shape[0] == A.shape[1]  # must be square matrix
    
    A = A.astype(float)  # ensure floating-point, create copy by default
    
    n = A.shape[0]
    w       = np.ones(n)
    lam     = np.linalg.norm(w)
    v       = w / lam
    lam_old = lam

    for i in range(max_iter):
        w   = A @ v  # matrix multiplication
        lam = np.linalg.norm(w)
        v   = w / lam
        if abs(lam - lam_old) < tol:
            print(f"Power method converged in {i+1} iterations.")
            return lam, v        
        lam_old = lam
        
    return lam, v

In [ ]:
# Test the Power Method
A = np.array([[2, 1],
              [1, 3]], dtype=float)

lam_dom, v_dom = eig_power(A)
print(lam_dom, v_dom, "power")

# Compare with NumPy
lams, vs = np.linalg.eig(A)
idx = np.argmax(np.abs(lams))
print(lams[idx], vs[:, idx], "numpy")

print('Testing:', A @ v_dom - lam_dom * v_dom)

In [ ]:
# HANDSON
#
# Try the power method with larger matrices.
# How fast/slow it is for convergence?
# How does the result compared to `numpy.linalg.eig()`?


### QR Algorithm for Dense Matrices

For dense matrices of moderate size, the $QR$ algorithm (or related
methods) is the workhorse for computing all eigenvalues and optionally
eigenvectors.

1. Idea:
   * Repeatedly factor $A$ as $QR$, where $Q$ is orthonormal and $R$
     is upper triangular.
   * Set $A \leftarrow RQ$.
   * After enough iterations, $A$ becomes near-upper-triangular,
     revealing the eigenvalues on its diagonal.

2. Practical Approach
   * In Python, use `numpy.linalg.eig` or `numpy.linalg.eigh` (for
     Hermitian matrices).
   * These functions wrap optimized LAPACK routines implementing $QR$
     or related transformations, e.g., divide-and-conquer, multiple
     relatively robust representations (MRRR).

In [ ]:
A = np.array([[4,  1,  2],
              [1,  3,  1],
              [2,  1,  5]], dtype=float)

lams, vs = np.linalg.eig(A)
print('Eigenvalues:', lams)
print('Eigenvectors:')
print(vs)

print('Testing:')
for i in range(vs.shape[1]):
    print(A @ vs[:,i] - lams[i] * vs[:,i])

## Application: Coupled Harmonic Oscillators

Harmonic oscillators are a classic problem in physics, in this
hands-on, we will:
1. Derive or reference the analytical solution for two coupled
   oscillators.
2. Numerically solve the same system (using an eigenvalue approach).
3. Generalize to $n$ (and even $n \times n$) coupled oscillators,
   visualizing the mode shapes.

### Two Coupled Oscillators--Analytical Solution

Consider two masses $m$ connected by three springs (constant $k$),
arranged in a line and connected to two walls:
```
|--k--[m]--k--[m]--k--|
```

If each mass can move only horizontally, the equations of motion form
a $2 \times 2$ eigenvalue problem.
Let:
* $x_1(t)$ be the horizontal displacement of **Mass 1** from its
  equilibrium position.
* $x_2(t)$ be the horizontal displacement of **Mass 2**.
We assume **small oscillations**, so Hooke’s law applies linearly.

* Mass 1 experiences:
  * A restoring force $-k \,x_1$ from the left wall spring.
  * A coupling force from the middle spring:
    if $x_2 > x_1$, that spring pulls Mass 1 to the right;
    if $x_1 > x_2$, it pulls Mass 1 to the left.
    The net contribution is $-k (x_1 - x_2)$.
  Summing forces (Newton's second law) gives:
  \begin{align}
    m \ddot{x}_1 = -k x_1 - k (x_1 - x_2).
  \end{align}
* Mass 2 experiences:
  * A restoring force $-k\,x_2$ from the right wall spring.
  * The coupling force from the middle spring: $-k(x_2 - x_1)$.
  Hence,
  \begin{align}
    m \ddot{x}_2 = -k x_2 - k (x_2 - x_1).
  \end{align}

Rewrite each equation:
\begin{align}
\begin{cases}
  m \ddot{x}_1 + 2k x_1 -  k x_2 = 0,\\
  m \ddot{x}_2 -  k x_1 + 2k x_2 = 0.
\end{cases}
\end{align}

We can write
$\mathbf{x} = \begin{pmatrix}x_1 \\ x_2\end{pmatrix}$
and express the system as:

\begin{align}
  m \ddot{\mathbf{x}} + K \mathbf{x} = \mathbf{0},
\end{align}
where
\begin{align}
  m \,\ddot{\mathbf{x}} = m \begin{pmatrix}\ddot{x}_1 \\[6pt] \ddot{x}_2\end{pmatrix}, \quad
  K = \begin{pmatrix}
    2k & -k \\
    -k & 2k
  \end{pmatrix}.
\end{align}

Equivalently,
\begin{align}
  \ddot{\mathbf{x}} + \frac{1}{m}\,K \,\mathbf{x} = \mathbf{0}.
\end{align}
This is a **second-order linear system** describing small
oscillations.

We look for solutions of the form
\begin{align}
  \mathbf{x}(t) = \mathbf{x}(0)\, e^{\,i\,\omega\,t},
\end{align}
where $\mathbf{x}(0)$ is the initial condition and $\omega$ is the
(angular) oscillation frequency.

Plugging into $m \ddot{\mathbf{x}} + K \mathbf{x} = 0$ gives:
\begin{align}
  -m \omega^2 \mathbf{X} + K \mathbf{X} = \mathbf{0}
  \quad \Longrightarrow \quad
  \left(K - m \omega^2 I\right) \mathbf{X} = \mathbf{0}.
\end{align}
Nontrivial solutions exist only if
\begin{align}
  \det(K - m \omega^2 I) = 0,
\end{align}
which is the **eigenvalue problem** for $\omega^2$.

Explicitly, $K - m \omega^2 I$ is:
\begin{align}
\begin{pmatrix}
  2k - m \omega^2 & -k \\
  -k & 2k - m \omega^2
\end{pmatrix}.
\end{align}

The determinant is $(2k - m \omega^2)^2 - k^2$.
Setting this to zero results 
\begin{align}
  2k - m \omega^2 = \pm k.
\end{align}

Hence, we get **two** solutions for $\omega^2$:

1. $\omega_+^2$: taking the $+$ sign:
   \begin{align}
     2k - m \omega_+^2
     = k \quad \Longrightarrow \quad \omega_+^2
     = \frac{k}{m} \quad \Longrightarrow \quad \omega_1
     = \sqrt{\frac{k}{m}}.
   \end{align}

2. $\omega_-^2$: taking the $-$ sign:
   \begin{align}
     2k - m \omega_-^2
     =-k \quad \Longrightarrow \quad \omega_-^2
     = \frac{3k}{m} \quad \Longrightarrow \quad \omega_2
     = \sqrt{\frac{3k}{m}}.
   \end{align}

For each of the normal modes:

* Lower Frequency $\omega_+ = \sqrt{k/m}$:
  Plug $\omega_+^2 = k/m$ back into $(K - m\,\omega_+^2 I)\mathbf{X} = 0$.
  For instance,
  \begin{align}
    \begin{pmatrix}
    2k - k & -k \\
    -k & 2k - k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \begin{pmatrix}
    k & -k \\
    -k & k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \mathbf{0}.
  \end{align}
  This implies $x_1 = x_2$.
  Physically, the **in-phase** mode has both masses moving together.

* **Higher Frequency** $\omega_- = \sqrt{3k/m}$:
  \begin{align}
    \begin{pmatrix}
    2k - 3k & -k \\
    -k & 2k - 3k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \begin{pmatrix}
    -k & -k \\
    -k & -k
    \end{pmatrix}
    \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
    = \mathbf{0}.
  \end{align}
  This yields $x_1 = -x_2$.
  Physically, the **out-of-phase** mode has the two masses moving in opposite directions.

We can compute the position of these coupled oscillators according to
the analytical solutions.

In [ ]:
# Physical constants
m = 1.0  # mass
k = 1.0  # spring constant

# Frequencies for two normal modes
omegap = np.sqrt(k/m)      # in-phase
omegam = np.sqrt(3*k/m)    # out-of-phase

# Initial conditions
x1_0 = 0
x2_0 = 0.5

# The analytical solution:
def X_analytic(t):
    xp_0 = (x1_0 + x2_0) / 2
    xm_0 = (x1_0 - x2_0) / 2

    xp = xp_0 * np.cos(omegap * t)
    xm = xm_0 * np.cos(omegam * t)

    return xp + xm, xp - xm

Plot multiple frames:

In [ ]:
from pathlib import Path

import matplotlib.pyplot    as plt
import matplotlib.animation as animation

def mkplots(X, t_max=10, n_frames=201):
    T = np.linspace(0, t_max, n_frames)

    Path("plots").mkdir(parents=True, exist_ok=True)
    for i, t in enumerate(T):
        x1, x2 = X(t)
        plt.plot([-2,-1+x1,1+x2,2], [0,0,0,0], 'o-')
        plt.xlim(-2,2)
        plt.savefig(f"plots/{i:04}.png")
        plt.close()

mkplots(X_analytic)

In [ ]:
! rm -rf movie.mpg && ffmpeg -i plots/%04d.png -qmax 2 movie.mpg

### Two Coupled Oscillators--Semi-Analytical/Numerical Solution

Instead of solving the coupled oscillator problem analytically, we can
at least solve the eigenvalue part of the problem numerically.

In [ ]:
# HANDSON: Step 1. rewrite the analytical solution in matrix form

# Physical constants
m = 1.0  # mass
k = 1.0  # spring constant

# Frequencies for two normal modes
Omega = np.array([...])  # this should become a numpy array

# Initial conditions
X0 = np.array([...])  # this should become a numpy array

# The analytical solution in matrix notation:
def X_matrix(t):
    M0 = ...  # apply an transformation to rewrite the transformation in terms of eigenvectors
    M  = M0 * np.cos(Omega * t)
    return ...  # apply an inverse transformation to rewrite the modes in terms of x1 and x2

In [ ]:
# HANDSON: Step 2. Replace the manual solutions of eigenvalues Omega
#          and the transform by calling `numpy.linalg.eig()`

# Coupling matrix
K = np.array([
    [...],
    [...],
])

# Initial conditions
X0 = np.array([...])

# The analytical solution in matrix notation:
def X_matrix(t):
    ...
    Omega = ...
    M0    = ...
    M     = M0 * np.cos(Omega * t)
    return ...

In [ ]:
# HANDSON: Step 3. Generalize the solution to work for arbitrary
#          number of coupled oscillators

# Coupling matrix
K = np.array([
    [...],
    [...],
])

# Initial conditions
X0 = np.array([...])

# The analytical solution in matrix notation:
def X_matrix(t):
    ...
    Omega = ...
    M0 = ...
    M = M0 * np.cos(Omega * t)
    return ...

In [ ]:
# HANDSON: Step 4. Turn the outputs into a movie

# mkplots(X)

## Future Topics

### Singular Value Decomposition (SVD)

[SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition)
factorizes $A = U \Sigma V^T$.
It reveals how a matrix stretches vectors in orthogonal directions.
It works for non-square matrices, provides optimal low-rank
approximations, is numerically stable, and is widely used for
pseudoinverses, compression, and noise reduction.

### Principal Component Analysis (PCA)

[PCA](https://en.wikipedia.org/wiki/Principal_component_analysis)
applies SVD to centered data to find directions of maximum variance.
It reduces dimensionality, highlights dominant modes, and simplifies
noisy datasets, making it essential in physics, data modeling, and
machine learning.

SVD and PCA are cores of dimensionality reduction.
We may revisit them when we connect linear algebra to machine
learning.